307__1_digitaltmuseum.ipynb


In [1]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Start: 2025-11-09 14:57:21


In [7]:
import requests
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from concurrent.futures import ThreadPoolExecutor
from SPARQLWrapper import SPARQLWrapper, JSON
from IPython.display import display, HTML
from tqdm.notebook import tqdm
import re
import time

# -------------------------------
# 1️⃣ Hämta SAT-relaterade platser från Wikidata
# -------------------------------
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery("""
SELECT DISTINCT ?item ?itemLabel ?coord
(CONCAT(
  "https://api.dimu.org/api/solr/select?",
  "q=", ENCODE_FOR_URI(?itemLabel),
  "&fq=(artifact.defaultMediaIdentifier:* AND artifact.hasMedia:true)",
  "&fl=artifact.ingress.title,artifact.ingress.description,artifact.ingress.license,artifact.defaultMediaIdentifier,identifier.owner",
  "&rows=1000&wt=json&api.key=demo"
) AS ?api)
WHERE {
  {?item (wdt:P361|wdt:P131|wdt:P706|wdt:P138) wd:Q131318799.
  }
  UNION
  {
    ?item wdt:P6104 wd:Q134294510.
  }
  OPTIONAL { ?item wdt:P625 ?coord. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
ORDER BY ?itemLabel
""")
sparql.setReturnFormat(JSON)
data = sparql.query().convert()

places = []
for r in data["results"]["bindings"]:
    coord = r.get("coord", {}).get("value")
    label = r["itemLabel"]["value"]
    if coord and len(label) > 2:
        places.append({
            "item": r["item"]["value"],
            "label": label,
            "coord": coord,
            "api": r["api"]["value"]
        })
df_places = pd.DataFrame(places)
print(f"🔹 {len(df_places)} platser med koordinater kopplade till SAT")

# -------------------------------
# 2️⃣ Hjälpfunktioner
# -------------------------------
def normalize_license(l):
    if not l: return "(saknas)"
    l = l.lower()
    if "by-nc" in l: return "CC BY-NC"
    if "by-sa" in l: return "CC BY-SA"
    if "pdm" in l: return "CC PDM"
    if "by" in l: return "CC BY"
    if "ac" in l: return "AC"
    return "(saknas)"

def coord_to_latlon(coord_str):
    m = re.match(r"Point\\(([-0-9.]+) ([-0-9.]+)\\)", coord_str)
    if m:
        lon, lat = float(m.group(1)), float(m.group(2))
        return lat, lon
    return None, None

# -------------------------------
# 3️⃣ Hämta bilder från DigitaltMuseum (parallellt)
# -------------------------------
found_count = 0  # räknare för platser med bilder

def fetch_dimu(row):
    global found_count
    results = []
    try:
        r = requests.get(row["api"], timeout=10)
        data = r.json()
        print (row["api"])
        docs = data.get("response", {}).get("docs", [])
        if docs:
            found_count += 1
        for d in docs:
            print (d)
            lic = ", ".join(d.get("artifact.ingress.license", []) or ["(saknas)"])
            img_id = d.get("artifact.defaultMediaIdentifier")
            img_url = f"https://api.dimu.org/media/{img_id}?width=800" if img_id else None
            title = d.get("artifact.ingress.title", "(utan titel)")
            desc = d.get("artifact.ingress.description", "")
            results.append({
                "Plats": row["label"],
                "Wikidata": row["item"],
                "Titel": title,
                "Beskrivning": desc,
                "Licens": normalize_license(lic),
                "Bild": img_url,
                "Koord": row["coord"],
                "Länk": f"https://digitaltmuseum.se/search/?q={row['label']}"
            })
    except Exception:
        pass
    return results

t0 = time.time()
with ThreadPoolExecutor(max_workers=10) as ex:
    all_results = list(tqdm(ex.map(fetch_dimu, df_places.to_dict("records")), total=len(df_places)))

flat_results = [r for sublist in all_results for r in sublist]
df = pd.DataFrame(flat_results)
elapsed = time.time() - t0

print(f"🔹 {len(df)} bilder hittade totalt på {found_count} av {len(df_places)} platser")
print(f"⏱️ Körtid: {elapsed:.2f} sekunder")

# -------------------------------
# 4️⃣ Hantera tomma resultat
# -------------------------------
if df.empty:
    print("⚠️ Inga bilder hittades i DigitaltMuseum för dessa platser.")
else:
    # Summera bilder per licens
    license_summary = (
        df.groupby("Licens")["Titel"]
        .count()
        .reset_index()
        .rename(columns={"Titel": "Antal"})
        .sort_values("Antal", ascending=False)
    )
    display(HTML("<h3>🧾 Antal bilder per licens</h3>"))
    display(license_summary)

    # Logga platser utan bilder
    found_places = set(df["Plats"])
    missing = [p for p in df_places["label"] if p not in found_places]
    print(f"⚠️ {len(missing)} platser utan bilder")
    if missing:
        for m in missing:
            print(f" - {m}")

    # -------------------------------
    # 5️⃣ Skapa Folium-karta
    # -------------------------------
    m = folium.Map(location=[59.5, 18.8], zoom_start=8)
    cluster = MarkerCluster().add_to(m)

    for place, g in df.groupby("Plats"):
        lat, lon = coord_to_latlon(g.iloc[0]["Koord"])
        if lat and lon:
            n = len(g)
            lic_types = ", ".join(g["Licens"].unique())
            link = g.iloc[0]["Länk"]
            wd = g.iloc[0]["Wikidata"]
            example = g["Bild"].dropna().iloc[0] if g["Bild"].notna().any() else None

            html = f"""
            <div style='width:260px'>
            <b>{place}</b><br>
            {n} bilder<br>
            <b>Licenser:</b> {lic_types}<br>
            <a href='{link}' target='_blank'>DigitaltMuseum</a> |
            <a href='{wd}' target='_blank'>Wikidata</a><br>
            """
            if example:
                html += f"<img src='{example}' width='240' style='margin-top:5px;border-radius:8px;'>"
            html += "</div>"

            folium.Marker(
                [lat, lon],
                popup=folium.Popup(html, max_width=300),
                tooltip=f"{place} ({n} bilder)"
            ).add_to(cluster)

    m.save("SAT_bilder_licenser_bred_med_progress.html")
    display(m)


🔹 683 platser med koordinater kopplade till SAT


  0%|          | 0/683 [00:00<?, ?it/s]

https://api.dimu.org/api/solr/select?q=15%2C2%20cm-batteriet%20p%C3%A5%20Landsort&fq=(artifact.defaultMediaIdentifier:* AND artifact.hasMedia:true)&fl=artifact.ingress.title,artifact.ingress.description,artifact.ingress.license,artifact.defaultMediaIdentifier,identifier.owner&rows=1000&wt=json&api.key=demo
https://api.dimu.org/api/solr/select?q=Ahlmansvikens%20badplats&fq=(artifact.defaultMediaIdentifier:* AND artifact.hasMedia:true)&fl=artifact.ingress.title,artifact.ingress.description,artifact.ingress.license,artifact.defaultMediaIdentifier,identifier.owner&rows=1000&wt=json&api.key=demo
https://api.dimu.org/api/solr/select?q=Aff%C3%A4rshuset%2C%20Ut%C3%B6&fq=(artifact.defaultMediaIdentifier:* AND artifact.hasMedia:true)&fl=artifact.ingress.title,artifact.ingress.description,artifact.ingress.license,artifact.defaultMediaIdentifier,identifier.owner&rows=1000&wt=json&api.key=demo
https://api.dimu.org/api/solr/select?q=Ahlmansvikens%20g%C3%A4sthamn&fq=(artifact.defaultMediaIdentifier:*

In [3]:
df 

""


In [4]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
minutes, seconds = divmod(elapsed_time, 60)
print("Total time elapsed: {:02.0f} minutes {:05.2f} seconds".format(minutes, seconds))


Date: 2025-11-09 14:57:48
Total time elapsed: 00 minutes 26.59 seconds
